# QE input script generator
This will automate the process of writing input scripts for Quantum Espresso to do the structural optimization of functionalized mofs using the pw.x (plane wave) package and pseudopotential files. 

**NOTE:** many default arguements will be preset.

In [6]:
using PorousMaterials
using Printf

In [7]:
which_crystal = "NiPyC2"
replication_id = "sc211_" # super cell 2x1x1
qe_run_type = "relax"

# name of the directory containing functionalixed structures
mof_dir_name = which_crystal * "_" * replication_id * qe_run_type

@eval PorousMaterials PATH_TO_CRYSTALS = joinpath(pwd(), "mof_construction",
    "NiPyC2_sc211_relax") 

"/home/ng/DTRA/mof_construction/NiPyC2_sc211_relax"

In [8]:
# # The name of the fragment
# # file must be a .xyz file located in ./fragments/
# # The atom species that is bonded to the :C_aro_R atom on the ring segment of the Fragment
# fragment_list = ["Br", "CH2-CH2-CH3", "CH2-CH3", "CH3", 
#                 "Cl", "C-N", "F", "NH2", "N-NH", 
#                 "O-CH2-CH2-CH3", "O-CH2-CH3", "O-CH3", "OH"]

# # The type of Arene Substitution ("ortho", "meta", "all") 
# # TODO: "all" not yet implimented
# substitution_types = ["ortho", "meta"]

In [9]:
# base_name = "NiPyC2_relax"
# xtals = Dict{String, Crystal}()

# for frag in fragment_list
#     for sub_typ in substitution_types
#         xtal_name = base_name * "_" * sub_typ * "_functionalized_" * frag
#         xtals[xtal_name] = Crystal(xtal_name * ".cif")
#     end
# end

In [11]:
# import the functionalized mofs 
crystal = Crystal("NiPyC2_relax_meta_functionalized_NH2.cif")
strip_numbers_from_atom_labels!(crystal)

In [23]:
crystal

0

In [30]:
# create a dictionary consisting of input arguements
# needed for the script and set default values
# then populate with values specific to each material
# link with full list and description of input params
# https://www.quantum-espresso.org/Doc/INPUT_PW.html#__top__

input_params = Dict{String, Any}()
# &control params
input_params["crystal"] = crystal
input_params["calculation"] = "relax" 
input_params["prefix"] = split(crystal.name,".")[1]
input_params["pseudo_dir"] = joinpath(pwd(), "structural_relaxation", "pseudo")
input_params["outdir"] = joinpath(pwd(), "structural_relaxation")
input_params["nstep"] = 200

# &SYSTEM params
input_params["ibrav"] = 0
input_params["nat"] = crystal.atoms.n
input_params["ntyp"] = 6 # treat Ni specially
input_params["tot_charge"] = crystal.charges.n
input_params["ecutwfc"] = 100.0
input_params["vdw_corr"] = "grimme-d2"
input_params["occupations"] = "smearing"
input_params["degauss"] = 0.2
input_params["smearing"] = "mv"
input_params["nspin"] = 2
input_params["sub_lat_species_index"] = ["1", "2"]
input_params["starting_magnetization"] = [0.25, 0.25]
input_params["tot_magnetization"] = 8 # check calculation

# &ELECTRON params
input_params["electron_maxstep"] = 90
input_params["mixing_mode"] = "local-TF"
input_params["diagonalizatio"] = "david"

# &IONS params
input_params["ion_dynamics"] = "bfgs"

# &CELL params
input_params["cell_dynamics"] = "bfgs"
input_params["cell_dofree"] = "xyz"

# K_POINTS params
input_params["k_points"] = ["{gamma}"]

1-element Array{String,1}:
 "{gamma}"

In [31]:
input_params["sub_lat_species_index"][1]

1

In [5]:
# populate dictionary with names of pseudopotential files
# note: may need to ignore case sensitivity?
# ex: atomic_psp[:Ni] = ["ni_pbesol_v1.4.uspp.F.UPF"]

ps_lib = Dict{Symbol, String}()
ps_lib

Dict{Symbol,String} with 0 entries

In [30]:
filename = "pw." * crystal.name * ".in"

"pw.NiPyC2_relax_meta_functonalized_NH2.cif.in"

In [32]:
qe_input_file = open(filename, "w")

# write &Control
@printf(qe_input_file,
    """
    &CONTROL
       calculation = '%s',
       prefix = '%s',
       pseudo_dir = '%s',
       outdir = '%s'
       nstep = %.0f,
    /
    """,
    input_params["crystal"].name, input_params["crystal"].name,
    input_params["prefix"], input_params["pseudo_dir"],
    input_params["outdir"], input_params["nstep"]
)

# write &SYSTEM
@printf(qe_input_file,
    """
    &SYSTEM
       ibrav = %.0f,
       nat = %.0f,
       ntyp = %0.f,
       tot_charge = %f,
       ecutwfc = %f,
       vdw_corr = '%s',
       occupations = '%s',
       degauss = %f,
       smearing = '%s',
       nspin = %.0f ,
       starting_magnetization(%s) = %f,
       starting_magnetization(%s) = %f,
       tot_magnetization = %f
    /
    """, 
    input_params["ibrav"], input_params["nat"],
    input_params["ntyp"], input_params["tot_charge"],
    input_params["ecutwfc"], input_params["vdw_corr"],
    input_params["occupations"], input_params["degauss"],
    input_params["smearing"], 
    input_params["sub_lat_species_index"][1],
    input_params["starting_magnetization"][1],
    input_params["sub_lat_species_index"][2],
    input_params["starting_magnetization"][2],
    input_params["tot_magnetization"]
)

# write &ELECTRONS
@printf(qe_input_file,
    """
    &ELECTRONS
    electron_maxstep = %0.f,
    mixing_mode = '%s',
    diagonalization = '%s',
    /
    """,
    90, "local-TF", "david"
)

# write &IONS
printf(qe_input_file,
    """
    &IONS
    ion_dynamics = '%s',
    /
    """,
    "bfgs"
)


# write &CELL
printf(qe_input_file,
    """
    &CELL
    cell_dynamics = '%s'
    cell_dofree = '%s'
    /
    """,
    "bfgs", "xyz"
)

# write ATOMIC_SPECIES
printf(qe_input_file,
    """
    ATOMIC_SPECIES
    %s    %f    %s
    """,
    variables
)

# write K_POINTS
printf(qe_input_file,
    """
    K_POINTS %s
    """,
    "{gamma}"
)


# write CELL_PARAMETERS
printf(qe_input_file,
    """
    CELL_PARAMETERS %s
       %f   %f   %f
       %f   %f   %f
       %f   %f   %f
    """,
    "{angstrom}",
)

# write ATOMIC_POSITIONS
#symbol #x #y #z
ni_count = 1
for a = 1:crystal.atoms.n
    if crystal.atoms.species[a] == :Ni
        ni_count += 1
    end
    @printf("%s    %f  %f  %f\n",
        crystal.atoms.species[a], 
        crystal.atoms.coords.xf[1, a], 
        crystal.atoms.coords.xf[2, a], 
        crystal.atoms.coords.xf[3, a]
    )
end


UndefVarError: UndefVarError: n_count not defined

In [ ]:
close(qe_input_file)

run(`cat $filename`)

In [8]:
# # use %s for strings, 

# @printf(f,
#     """
#     &CONTROL
#     calculation = 'relax'
#     prefix = '%s'
#     pseudo_dir = 'pseudo/'
#     ! etot_conv_thr=1.0d-6,
#     ! 
#     ! wf_collect=.false.
#     /
#     &SYSTEM
#     ibrav = 0,
#     nat = %.0f,
#     ntyp = %.0f,
#     tot_charge = %f,
#     ecutwfc = %f,
#     vdw_corr='grimme-d2'
#     /
#     &ELECTRONS
#     diagonalization='david',
#     /
#     &IONS
#     ion_dynamics='bfgs',
#     /
#     &CELL
#     cell_dynamics='bfgs'
#     cell_dofree='xyz'
#     /
#     ATOMIC_SPECIES
#     #symbol #atomic_mass #pseudo
#     CELL_PARAMETERS (angstrom)
#     # matrix
#     ATOMIC_POSITIONS
#     #symbol #x #y #z
#     K_POINTS {gamma}
#     """,
    
#     args)

LoadError: ArgumentError: @printf: wrong number of arguments (1) should be (5)

In [9]:
close(f)

UndefVarError: UndefVarError: f not defined

In [10]:
# how do I write the cell parameters and
# atomic positions as a formatted string?